In [1]:
!pip install --upgrade pip
!pip install transformers
!pip install datasets>=1.18.3
!pip install librosa
!pip install jiwer
!pip install evaluate>=0.30



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 14.1 MB/s eta 0:00:00


In [2]:
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git@main
!pip install huggingface_hub
!pip install -U datasets huggingface-hub

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.4 MB/s eta 0:00:00


In [3]:
from scipy.io import wavfile
import scipy
from IPython.core.display import display
import os
import random
import IPython
import soundfile
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
import json
import time
from huggingface_hub import notebook_login
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from datasets import load_dataset, DatasetDict, Audio, load_metric
from transformers import WhisperTokenizer, WhisperProcessor, WhisperFeatureExtractor
from jiwer import wer

from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import gc

In [4]:
#utilities

notebook_login()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_wav_duration(audio_array, sample_rate):
    length = len(audio_array) / sample_rate

    return int(length)

In [6]:
from datetime import datetime as dt
import time
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset
import soundfile
import librosa
import numpy as np


class WhisperModel:
  def __init__(self, check_point):
    ckpt = check_point # "openai/whisper-medium" #"openai/whisper-tiny" #"vasista22/whisper-hindi-medium"
    self.processor = WhisperProcessor.from_pretrained(ckpt)
    self.model = WhisperForConditionalGeneration.from_pretrained(ckpt,)
    self.model.config.forced_decoder_ids = None
    self.model.to(device)
    self.initialize_supplements(check_point)

  def predict_with_whisper(self, audio_array, sample_rate):
    try:
      audio_array = librosa.resample(np.mean(audio_array, axis=1), orig_sr=sample_rate, target_sr=16000)
    except:
      audio_array = librosa.resample(audio_array, orig_sr=sample_rate, target_sr=16000)

    sample = {}
    sample["array"] = audio_array
    sample["sampling_rate"] = 16000
    input_features = self.processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").to(device).input_features
    # generate token ids

    predicted_ids = self.model.generate(input_features)

    transcription = self.processor.batch_decode(predicted_ids, skip_special_tokens=True)

    return transcription[0]

  def initialize_supplements(self, check_point):
    self.tokenizer = WhisperTokenizer.from_pretrained(check_point, language="en", task="transcribe")
    self.processor = WhisperProcessor.from_pretrained(check_point, language="en", task="transcribe")
    self.feature_extractor = WhisperFeatureExtractor.from_pretrained(check_point)

In [7]:
# class TranscriptGenerator:
#   def __init__(self, model):
#     self.speech_model = model

#   def generate_predictions(self, audio_array, sample_rate):
#     transcription = self.speech_model.predict_with_whisper(audio_array, sample_rate)

#     if get_wav_duration(audio_array, sample_rate)<1:
#       if len(transcription)>8:
#         return ["Ignore No meaningful Audio Present"]

#     return transcription

In [8]:
# check_point = "openai/whisper-medium"
# model = WhisperModel(check_point)

In [9]:

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [10]:
class ASRDataset:
  def __init__(self, tokenizer, processor, feature_extractor):
    self.dataset_path = "yashtiwari/PaulMooney-Medical-ASR-Data"
    self.train_data, self.validation_data, self.test_data = None, None, None
    self.data = DatasetDict()
    self.preprocessor = DataPreprocessing(tokenizer, processor, feature_extractor)
    self.data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=self.preprocessor.processor)

  def load_dataset(self):

    self.data["train"] = load_dataset(self.dataset_path, split="test", use_auth_token=True)
    self.data["validation"] = load_dataset(self.dataset_path, split="validation", use_auth_token=True)
    self.data["test"] = load_dataset(self.dataset_path, split="test", use_auth_token=True)

    print(f"data is loaded successfully!! \n {self.data}")

  def preprocess_data(self):
    self.data = self.data.remove_columns(
    ["prompt", "speaker_id", "id"])
    self.data = self.data.cast_column("path", Audio(sampling_rate=16000))

    #action
    self.data = self.data.map(self.preprocessor.prepare_dataset, remove_columns = self.data.column_names["train"], num_proc=1)

    print(f"preprocessing completed !!!")
    print(f"dataset structure looks like: {self.data['train']}")

class DataPreprocessing:
  def __init__(self, tokenizer, processor, feature_extractor):
    self.tokenizer = tokenizer
    self.processor = processor
    self.feature_extractor = feature_extractor

  def prepare_dataset(self, batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["path"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = self.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = self.tokenizer(batch["sentence"]).input_ids
    return batch



In [11]:

# class WERMetric:
#   def __init__(self):
#     self.wer = wer
#     self.score_list = []

#   def add_batch(self, predictions, references):
#     self.compute_metrics

#   def compute_metrics(self, pred):
#       pred_ids = pred.predictions
#       label_ids = pred.label_ids

#       # replace -100 with the pad_token_id
#       label_ids[label_ids == -100] = model.tokenizer.pad_token_id

#       # we do not want to group tokens when computing the metrics
#       pred_str = model.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
#       label_str = model.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

#       wer_score = 0
#       for pred, label in zip(pred_str, label_str):
#         wer_score += 100 * self.metric(predictions=pred, references=label)

#       return {"wer": wer_score}

In [12]:
# metric = load_metric("wer")
# def compute_metrics(pred):
#     pred_ids = pred.predictions
#     label_ids = pred.label_ids

#     # replace -100 with the pad_token_id
#     label_ids[label_ids == -100] = model.tokenizer.pad_token_id

#     # we do not want to group tokens when computing the metrics
#     pred_str = model.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
#     label_str = model.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

#     wer = 100 * metric.compute(predictions=pred_str, references=label_str)

#     return {"wer": wer}

In [13]:
# med_asr = ASRDataset(model.tokenizer, model.processor, model.feature_extractor)
# med_asr.load_dataset()

In [14]:
# med_asr.preprocess_data()

In [15]:
# from torch.utils.data import DataLoader
# from tqdm import tqdm
# import numpy as np
# import gc

# eval_dataloader = DataLoader(med_asr.data["test"], batch_size=8, collate_fn=med_asr.data_collator)

# model.model.eval()
# for step, batch in enumerate(tqdm(eval_dataloader)):
#     with torch.cuda.amp.autocast():
#         with torch.no_grad():
#             generated_tokens = (
#                 model.model.generate(
#                     input_features=batch["input_features"].to("cuda"),
#                     decoder_input_ids=batch["labels"][:, :4].to("cuda"),
#                     max_new_tokens=255,
#                 )
#                 .cpu()
#                 .numpy()
#             )
#             labels = batch["labels"].cpu().numpy()
#             labels = np.where(labels != -100, labels, model.tokenizer.pad_token_id)
#             decoded_preds = model.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
#             decoded_labels = model.tokenizer.batch_decode(labels, skip_special_tokens=True)
#             metric.add_batch(
#                 predictions=decoded_preds,
#                 references=decoded_labels,
#             )
#     del generated_tokens, labels, batch
#     gc.collect()
# wer = 100 * metric.compute()
# print(f"{wer=}")

In [18]:

class Report:
  def __init__(self, model_type, model_version):
    if model_type == "default":

      if model_version == "medium":
        check_point = "openai/whisper-medium"
      elif model_version == "tiny":
        check_point = "openai/whisper-tiny"
      elif model_version == "large":
        check_point = "openai/whisper-large-v2"
      else:
        raise ValueError("Please input a valid model_version name from ['medium', 'tiny', 'large']")
    elif model_type == "fine-tuned":
      pass

    else:
        raise ValueError("Please input a valid model_type  from ['default', 'fine-tuned']")

    self.whisper_model = WhisperModel(check_point)
    self.dataset_path = "yashtiwari/PaulMooney-Medical-ASR-Data"
    self.data, self.check_point = None, check_point

    #initialize
    self.initialize_supplements()


  def initialize_supplements(self):
    self.metric = load_metric("wer")

    #prepare dataset
    self.med_asr = ASRDataset(self.whisper_model.tokenizer, self.whisper_model.processor, self.whisper_model.feature_extractor)
    self.med_asr.load_dataset()
    self.med_asr.preprocess_data()

  def compute_metrics(self, pred):
      pred_ids = pred.predictions
      label_ids = pred.label_ids
      label_ids[label_ids == -100] = self.whisper_model.tokenizer.pad_token_id
      pred_str = self.whisper_model.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
      label_str = self.whisper_model.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
      wer = 100 * self.metric.compute(predictions=pred_str, references=label_str)

      return {"wer": wer}

  def process_data(self, split = "test"):
    eval_dataloader = DataLoader(self.med_asr.data[split], batch_size=8, collate_fn=self.med_asr.data_collator)

    self.whisper_model.model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        with torch.cuda.amp.autocast():
            with torch.no_grad():
                generated_tokens = (
                    self.whisper_model.model.generate(
                        input_features=batch["input_features"].to("cuda"),
                        decoder_input_ids=batch["labels"][:, :4].to("cuda"),
                        max_new_tokens=255,
                    )
                    .cpu()
                    .numpy()
                )
                labels = batch["labels"].cpu().numpy()
                labels = np.where(labels != -100, labels, self.whisper_model.tokenizer.pad_token_id)
                decoded_preds = self.whisper_model.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
                decoded_labels = self.whisper_model.tokenizer.batch_decode(labels, skip_special_tokens=True)
                self.metric.add_batch(
                    predictions=decoded_preds,
                    references=decoded_labels,
                )
        del generated_tokens, labels, batch
        gc.collect()
    wer = 100 * self.metric.compute()
    return wer

  def generate_report(self):

    validation_wer = self.process_data("validation")
    test_wer = self.process_data("test")

    print("="*30)
    print(f"dataset path: {self.dataset_path}")
    print(f"-"*30)
    print(f"model check point: {self.check_point}")
    print(f"-"*30)
    print(f"validation WER: {validation_wer} ")
    print(f"-"*30)
    print(f"test WER: {test_wer} ")
    print("="*30)




In [ ]:
if __name__ == "__main__":
  report = Report(model_type = "default", model_version = "medium")
  report.generate_report()

<ipython-input-18-cb04aecebaf1>:28: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  self.metric = load_metric("wer")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.0/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:2508: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
